# Using Azure PostgreSQL as the Unified Data Layer for an Agentic Framework with LangGraph

This notebook demonstrates an end-to-end pattern for using **Azure Database for
PostgreSQL** as the *single* persistence + retrieval layer for an AI agent graph
built with **LangGraph**.

We consolidate:

- Durable agent state / checkpoints
- Episodic & semantic memory
- Chat history (multi-session)
- Vector storage (pgvector) for Retrieval Augmented Generation (RAG)

## High-Level Architecture

| Concern                 | Postgres Object                 | Notes                                              |
| ----------------------- | ------------------------------- | -------------------------------------------------- |
| Agent graph checkpoints | checkpoints                     | Serialized LangGraph state per thread / step       |
| Chat history            | checkpoints                     | Ordered log of messages (roles, content, metadata) |
| Long-term memory        | store                           | Key-value or typed memory entries                  |
| Knowledge / docs        | documents (content + embedding) | Vector similarity over pgvector                    |
| Embeddings index        | documents.embedding (vector)    | Uses pgvector `vector` type                        |

We'll walk through: installation, connection, schema DDL, helper data-access functions, LangGraph integration, vector similarity search, and a retrieval-augmented interaction.

## ⚠️ Prerequisites

1. An Azure Database for PostgreSQL Flexible Server (v14+ recommended) with pgvector enabled.
2. `pgvector` extension installed (we show how).
3. API key for an embedding + chat model (Azure OpenAI or OpenAI).

Set environment variables before running (example):

```shell
export AZURE_PG_HOST=your-server.postgres.database.azure.com
export AZURE_PG_DB=agentdb
export AZURE_PG_USER=agent_user
export AZURE_PG_PASSWORD='YOUR_SECRET'  # consider Azure Key Vault in production
export OPENAI_API_KEY=...              # or Azure OpenAI specifics
# For Azure OpenAI also set: AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, AZURE_OPENAI_DEPLOYMENT
```

> Performance tip: Co-locate compute (where LangGraph runs) and your Azure
  Postgres server in the same region to minimize latency. Use connection pooling
  (e.g. PgBouncer / Azure Flexible Server built-in) for concurrency.

## Import Necessary Modules and Packages


In [25]:
# If environment variables from .env are not loaded/read automatically when the
# notebook kernel is (re)started (default behavior with Visual Studio Code and
# uv), then you can manually load them using the following code:

from dotenv import load_dotenv

load_dotenv()

True

## Connect to Azure PostgreSQL

We'll define the connection string or EntraID. Use SSL (`sslmode=require`) which
Azure enforces by default. For production, prefer a connection pool (e.g.,
PgBouncer) and secrets from Azure Key Vault instead of raw env vars.

In [26]:
import os

from azure.identity import DefaultAzureCredential
from langchain_azure_postgresql import (
    AzurePGConnectionPool,
    ConnectionInfo,
    BasicAuth,
    Extension,
    create_extensions,
    SSLMode,
)
from langchain_core.documents import Document
import numpy as np
from pgvector.psycopg import register_vector
from psycopg import Connection
from psycopg.rows import dict_row


In [27]:
PGHOST = os.getenv("PGHOST", "localhost")
connection_pool = AzurePGConnectionPool(
    azure_conn_info=ConnectionInfo(
        host=PGHOST,
    ),
)

# Test that the connection pool works
connection_pool.wait()

with connection_pool.connection() as conn, conn.cursor() as cur:
    cur.execute("select 1 as response")
    result = cur.fetchone()
    assert result is not None
    print(f"{result=}")

result=(1,)


### Enable pgvector and DiskANN Extension (if not already)

In [4]:
def configure_connection(conn: Connection) -> None:
    """Configure the PostgreSQL connection (as received from the pool).

    `PostgresSaver` requires that the connection is initialized with a
    `dict_row` row factory and set to auto-commit mode (cf. the docs).

    We will make sure `pg_diskann` is created (together with `vector`) when the
    connection is given (via `create extension if not exists pg_diskann cascade`).

    Finally, we will register pgvector loaders and dumpers.

    :param conn: The PostgreSQL connection to configure.
    :type conn: Connection
    """
    conn.autocommit = True
    conn.row_factory = dict_row
    create_extensions(conn, [Extension(ext_name="pg_diskann", cascade=True)])
    register_vector(conn)

### Using EntraID and Azure Postgres LangChain
The following sections contain functions that set up LangChain to use Microsoft Entra authentication.

To sign in to Azure, ensure that you have the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed. Run the following command in your terminal:

```bash
az login
```

In [28]:
PGHOST = os.getenv("PGHOST", "localhost")

connection_pool = AzurePGConnectionPool(
    azure_conn_info=ConnectionInfo(
        host=PGHOST
    ),
    configure=configure_connection,
)


# Using Postgres as a vector store

## Set up Azure OpenAI embeddings

In [29]:
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings

model = "text-embedding-3-small"

embedding = (
    AzureOpenAIEmbeddings(model=model)
    if os.getenv("AZURE_OPENAI_API_KEY") is not None
    else OpenAIEmbeddings(model=model)
)

In [30]:
# Test that embedding generation works
# For instance, OpenAI models have unit-norm embedding vectors

query_embedding = embedding.embed_query("Hello World")
assert np.isclose(np.linalg.norm(query_embedding), 1, atol=1e-4)

print(f"{type(query_embedding)=}, {len(query_embedding)=}")

type(query_embedding)=<class 'list'>, len(query_embedding)=1536


## Set Up Vector Store

We define a normalized schema. Adjust dimensions to match the embedding model you use (e.g. 1536 for `text-embedding-3-small`).

In [36]:
from langchain_azure_postgresql import AzurePGVectorStore

table_name = "langgraph"

# Get a connection from the pool to pass to the vector store.
# Ideally, this should be done in a context manager to ensure proper cleanup.
# For this notebook, though, we'll just manually release the connection.
conn_vectorstore = connection_pool.getconn()

vector_store = AzurePGVectorStore(
    embedding=embedding, connection=conn_vectorstore, table_name=table_name
)

In [33]:
docs = [
    Document(
        page_content="there are cats in the pond",
        metadata={"doc_id": 1, "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="ducks are also found in the pond",
        metadata={"doc_id": 2, "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="fresh apples are available at the market",
        metadata={"doc_id": 3, "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the market also sells fresh oranges",
        metadata={"doc_id": 4, "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the new art exhibit is fascinating",
        metadata={"doc_id": 5, "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a sculpture exhibit is also at the museum",
        metadata={"doc_id": 6, "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a new coffee shop opened on Main Street",
        metadata={"doc_id": 7, "location": "Main Street", "topic": "food"},
    ),
    Document(
        page_content="the book club meets at the library",
        metadata={"doc_id": 8, "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="the library hosts a weekly story time for kids",
        metadata={"doc_id": 9, "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="a cooking class for beginners is offered at the community center",
        metadata={"doc_id": 10, "location": "community center", "topic": "classes"},
    ),
]

uuids = vector_store.add_documents(docs)

In [ ]:
results = vector_store.similarity_search_with_score(query="something creative that will inspire", k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.645530] the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* [SIM=0.645548] the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* [SIM=0.645548] the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]


## Update items in the vector store

In [11]:
docs = [
    Document(
        page_content="Updated - a cooking class for beginners is offered at the community center",
        metadata={"doc_id": 11, "location": "community center", "topic": "classes"},
    )
]
vector_store.add_documents(docs, ids=[uuids[-1]], on_conflict_update=True)

['9629a3aa-2dab-44ca-a833-29bc2491e34f']

## Delete items from the vector store

In [12]:
vector_store.delete(ids=[uuids[1]])

True

### Direct query
You can perform a simple similarity search as follows:

In [13]:
from langchain_azure_postgresql import FilterCondition, AndFilter


In [14]:
results = vector_store.similarity_search(
    "kitty",
    k=10,
    filter=FilterCondition(
        column="(metadata->>'doc_id')::int",
        operator="in",
        value=[1, 3, 5, 7, 9],
    ),
)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]
* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]
* the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* the library hosts a weekly story time for kids [{'topic': 'reading', 'doc_id': 9, 'location': 'library'}]
* the library hosts a weekly story time for kids [{'topic': 'reading', 'doc_id': 9, 'location': 'library'}]
* fresh apples are available at the market [{'topic': 'food', 'doc_id': 3, 'location': 'market'}]
* fresh apples are available at the market [{'topic': 'food', 'doc_id': 3, 'location': 'market'}]
* a new coffee shop opened on Main Street [{'topic': 'food', 'doc_id': 7, 'location': 'Main Street'}]
* a new coffee shop opened on Main Street [{'topic': 'food', 'doc_id': 7, 'location': 'Main Street'}]


In [15]:
results = vector_store.similarity_search(
    "ducks",
    k=10,
    filter=AndFilter(
        AND=[
            FilterCondition(
                column="(metadata->>'doc_id')::int",
                operator="in",
                value=[1, 5, 2, 9],
            ),
            FilterCondition(
                column="metadata->>'location'",
                operator="in",
                value=["pond", "market"],
            ),
        ]
    ),
)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]
* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]


If you want to execute a similarity search and receive the corresponding scores, you can run:

In [16]:
results = vector_store.similarity_search_with_score(query="cats", k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.528251] there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]


### Transformation into a retriever
You can also transform the vector store into a retriever for easier usage in your chains:

## Filtered Search

In [17]:
results = vector_store.max_marginal_relevance_search(
    "query about cats",
    k=10,
    lambda_mult=0.5,
    filter=FilterCondition(
        column="(metadata->>'doc_id')::int",
        operator="in",
        value=[1, 3, 5, 7, 9],
    ),
)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]
* the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* fresh apples are available at the market [{'topic': 'food', 'doc_id': 3, 'location': 'market'}]
* the library hosts a weekly story time for kids [{'topic': 'reading', 'doc_id': 9, 'location': 'library'}]
* a new coffee shop opened on Main Street [{'topic': 'food', 'doc_id': 7, 'location': 'Main Street'}]
* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]
* fresh apples are available at the market [{'topic': 'food', 'doc_id': 3, 'location': 'market'}]
* the new art exhibit is fascinating [{'topic': 'art', 'doc_id': 5, 'location': 'museum'}]
* the library hosts a weekly story time for kids [{'topic': 'reading', 'doc_id': 9, 'location': 'library'}]
* a new coffee shop opened on Main Street [{'topic': 'food', 'doc_id': 7, 'location': 'Main Street'}]


In [18]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
results = retriever.invoke("kitty")

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* there are cats in the pond [{'topic': 'animals', 'doc_id': 1, 'location': 'pond'}]


# Using Postgres for Chat History & Memory Management

[Memory](https://langchain-ai.github.io/langgraph/how-tos/memory/add-memory/) is a system that remembers information about previous interactions. For AI agents, memory is crucial because it lets them remember previous interactions, learn from feedback, and adapt to user preferences. 

This conceptual guide covers two types of memory, based on their recall scope:

* [Short-term memory](https://langchain-ai.github.io/langgraph/concepts/memory/#short-term-memory), or [thread](https://langchain-ai.github.io/langgraph/concepts/persistence/#threads)-scoped memory, tracks the ongoing conversation by maintaining message history within a session. LangGraph manages short-term memory as a part of your agent's [state](https://langchain-ai.github.io/langgraph/concepts/low_level/#state). State is persisted to a database using a [checkpointer](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpoints). 

* [Long-term memory](https://langchain-ai.github.io/langgraph/concepts/memory/#long-term-memory) stores user-specific or application-level data across sessions and is shared across conversational threads. It can be recalled at any time and in any thread. LangGraph provides [stores](https://langchain-ai.github.io/langgraph/concepts/persistence/#memory-store) to let you save and recall long-term memories.

More details [here](https://langchain-ai.github.io/langgraph/concepts/memory/#memory)

## Store & Retrieve Agent Short Term Memory

We'll implement helper functions for writing semantic or structured memory, optionally embedding values for similarity recall.

In [19]:
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.store.postgres import PostgresStore


In [20]:
# Setup is done the first time checkpointer and store are used

with (
    connection_pool.connection() as conn_checkpointer,
    connection_pool.connection() as conn_store,
):
    checkpointer = PostgresSaver(conn_checkpointer)
    checkpointer.setup()

    store = PostgresStore(conn_store)
    store.setup()


In [21]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START

model = init_chat_model(model="azure_openai:gpt-4.1-mini")


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


conn_checkpointer = connection_pool.getconn()
checkpointer = PostgresSaver(conn_checkpointer)

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
graph = builder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! What can I do for you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob. How can I assist you today?


## Store & Retrieve Agent Long Term Memory

In [22]:
from langchain_core.runnables import RunnableConfig
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.store.postgres import PostgresStore
from langgraph.store.base import BaseStore
import uuid

conn_store = connection_pool.getconn()
store = PostgresStore(conn_store)


def call_model(
    state: MessagesState,
    config: RunnableConfig,
    *,
    store: BaseStore,
):
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # Store new memories if the user asks the model to remember
    last_message = state["messages"][-1]
    if "remember" in last_message.content.lower():
        memory = "User name is Bob"
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")

graph = builder.compile(
    checkpointer=checkpointer,
    store=store,
)

config = {
    "configurable": {
        "thread_id": "1",
        "user_id": "1",
    }
}
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "Hi! Remember: my name is Bob"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

config = {
    "configurable": {
        "thread_id": "2",
        "user_id": "1",
    }
}

for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "what is my name?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! Remember: my name is Bob
================================== Ai Message ==================================

Hi Bob! I remember your name. What would you like to do today?
================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

Your name is Bob. How can I assist you today, Bob?


## LangGraph Quick Start Demo using Postgres for Memory and Vector Search
Using Postgres for Short Term Memory and Vector Search

In [23]:
from langgraph.prebuilt import create_react_agent


def get_data_from_vector_store(query: str) -> str:
    """Get data from the vector store."""
    results = vector_store.similarity_search(query)
    return results


agent = create_react_agent(
    model=model, tools=[get_data_from_vector_store], checkpointer=checkpointer
)

# Run the agent
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What does my database say about cats? Make sure you address me with my name",
            }
        ]
    },
    config,
)


In [24]:
for message in response["messages"][-2:]:
    message.pretty_print()

================================ Human Message =================================

What does my database say about cats? Make sure you address me with my name
================================== Ai Message ==================================

Bob, your database says that cats are independent pets that often enjoy their own space. If you need more details or have other questions, just let me know!


## More Examples
We have another doc for Long Term Memory and Postgres here: ###### https://python.langchain.com/docs/versions/migrating_memory/long_term_memory_agent/

## Summary
We consolidated agent state, chat history, semantic memory, and vector search into Azure PostgreSQL, simplifying infra & governance. LangGraph's pluggable checkpointing lets Postgres serve as a durable orchestrator backend. pgvector enables RAG without an external vector DB.
